In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train_df=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_df=pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df.head()

In [ ]:
train_df.iloc[1,1:].values.reshape(28,28)

In [ ]:
fig=plt.figure(figsize=(10,10))
for i in range(1,10):
    x=np.random.randint(1000)
    fig.add_subplot(3,3,i)
    plt.title('Label: {}'.format(train_df.iloc[x,0]))  
    plt.imshow(train_df.iloc[x,1:].values.reshape(28,28),cmap='gray')  
    plt.colorbar()

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(data=train_df,x='label')


In [ ]:
for c in range(0,10):                                       # Range upto Number of class labels
    fig=plt.figure(figsize=(10,10))
    fig.suptitle('Sample Class: {}'.format(c))              # To place title at the centre of the image
    data=train_df[train_df.label==c]
    for i in range(1,10):
        fig.add_subplot(3,3,i)
        plt.title('Label: {}'.format(data.iloc[i,0]))  
        plt.axis('off')
        plt.imshow(data.iloc[i,1:].values.reshape(28,28),cmap='cool')

In [ ]:
X=train_df.iloc[:,1:].values                  # Pixels into X
Y=train_df.iloc[:,0]               

In [ ]:
from sklearn.preprocessing import MinMaxScaler
norm=MinMaxScaler()
norm.fit(X)
transnorm=norm.transform(X)

In [ ]:
X.shape,Y.shape

In [ ]:
from keras.utils import to_categorical
X=X.reshape(-1,28,28,1)
Y=to_categorical(Y,num_classes=10)


In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_valid,Y_train,Y_valid=train_test_split(X,Y,test_size=0.2,random_state=4)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator as ImgDataGen
augment_data= ImgDataGen(rescale=1./255,
                        rotation_range=20,
                         height_shift_range=0.2,
                         width_shift_range=0.2,
                         horizontal_flip=False,
                         zoom_range=0.10)

In [ ]:
from keras.models import Sequential
from keras.layers import Dropout,Conv2D,MaxPooling2D,Dense,Flatten,BatchNormalization

In [ ]:
model=Sequential()

model.add(Conv2D(128,(3,3),input_shape=(28,28,1),activation='relu',data_format='channels_last',name='conv1'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),activation='relu',name='conv2'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3),activation='relu',name='conv3'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),activation='relu',name='conv4'))
model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(32,(3,3),activation='relu',name='conv5'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),activation='relu',name='conv6'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(10,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
plot_model(model,to_file='model.png',show_shapes=True)

In [ ]:
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
earlystop=EarlyStopping(monitor="val_accuracy",min_delta=0,patience=10,mode='max',restore_best_weights=True)

In [ ]:
history=model.fit_generator(augment_data.flow(X_train, Y_train, batch_size = 64),steps_per_epoch = (len(X_train)*0.8 // 64),
 validation_data = augment_data.flow(X_valid, Y_valid), validation_steps=(len(X_valid)*0.2)//64,epochs = 100,callbacks=[earlystop])

In [ ]:
loss, acc = model.evaluate(augment_data.flow(X_valid, Y_valid, batch_size=64, seed=2))
print("Loss: {}\nAccuracy: {}".format(loss, acc))

In [ ]:
fig=plt.figure(figsize=(10,10))
for i in range(1,10):
    fig.add_subplot(3,3,i)
    x=np.random.randint(len(X_valid))
    plt.imshow(X_valid[x].reshape(28,28), cmap='cool')
    plt.axis('off')
    plt.title('True Label: {} and Predicted {}'.format(np.argmax(Y_valid[x]),np.argmax(model.predict(X_valid[x].reshape(-1,28,28,1)))))

In [ ]:
X_test = test_df.iloc[:, :].values

scaler = MinMaxScaler()
scaler.fit(X_test)
X_test = scaler.transform(X_test)

X_test = X_test.reshape(-1, 28, 28, 1)
print('Test Data Shape: ', X_test.shape)

In [ ]:
predictions = np.argmax(model.predict(X_test), axis=1)

In [ ]:
sub = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
sub['Label'] = predictions

sub.head()

In [ ]:
sub.to_csv('MNIST_using_CNN.csv', index=False)